## Step 1. Objectives of the Analysis

The objective of this analysis is to explore and visualize the meteorite landings data provided by NASA. The analysis aims to:

1. Understand the distribution and characteristics of meteorite landings.
2. Identify patterns or trends in the data.
3. Create interactive visualizations to present the findings in an engaging and informative way.


In [91]:
# !pip install ydata-profiling
# !pip install pandas_geojson
# !pip install geopandas

In [92]:
# Step 2. Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
import requests
from sqlalchemy import create_engine
import sqlite3
import plotly.express as px

import pandas_geojson as pdg
import geopandas as gpd
from pandas import json_normalize
import requests
import json

from ydata_profiling import ProfileReport

In [93]:
# Step 3. Load the dataset from NASA URL
url = 'https://data.nasa.gov/resource/gh4g-9sfh.json?$limit=50000'
response = requests.get(url)
data = response.json()

# Convert the data into a pandas DataFrame
# df = pd.DataFrame(data)
df = json_normalize(data)

## 4. Exploratory Data Analysis (EDA) and Data Cleaning

In [94]:
# 4.1 Initial exploration

# Display the first few rows of the dataframe
print("----------'Print the Head'-----------------")
display(df.head())

----------'Print the Head'-----------------


,name,id,nametype,recclass,mass,fall,year,reclat,reclong,geolocation.latitude,geolocation.longitude,:@computed_region_cbhk_fwbd,:@computed_region_nnqa_25f4
0,Aachen,1,Valid,L5,21,Fell,1880-01-01T00:00:00.000,50.775000,6.083330,50.775,6.08333,NaN,NaN
1,Aarhus,2,Valid,H6,720,Fell,1951-01-01T00:00:00.000,56.183330,10.233330,56.18333,10.23333,NaN,NaN
2,Abee,6,Valid,EH4,107000,Fell,1952-01-01T00:00:00.000,54.216670,-113.000000,54.21667,-113.0,NaN,NaN
3,Acapulco,10,Valid,Acapulcoite,1914,Fell,1976-01-01T00:00:00.000,16.883330,-99.900000,16.88333,-99.9,NaN,NaN
4,Achiras,370,Valid,L6,780,Fell,1902-01-01T00:00:00.000,-33.166670,-64.950000,-33.16667,-64.95,NaN,NaN


In [95]:
# Display basic information about the dataframe
print("----------'Print DataFrame Information'-----------------")
display(df.info())

----------'Print DataFrame Information'-----------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45716 entries, 0 to 45715
Data columns (total 13 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   name                         45716 non-null  object
 1   id                           45716 non-null  object
 2   nametype                     45716 non-null  object
 3   recclass                     45716 non-null  object
 4   mass                         45585 non-null  object
 5   fall                         45716 non-null  object
 6   year                         45425 non-null  object
 7   reclat                       38401 non-null  object
 8   reclong                      38401 non-null  object
 9   geolocation.latitude         38401 non-null  object
 10  geolocation.longitude        38401 non-null  object
 11  :@computed_region_cbhk_fwbd  1659 non-null   object
 12  :@computed_region_nnqa_25f4  16

None

In [96]:
# Display basic statistics of the dataframe
print("----------'Print the basic stats'-----------------")
display(df.describe().T)

----------'Print the basic stats'-----------------


,count,unique,top,freq
name,45716,45716,Aachen,1
id,45716,45716,1,1
nametype,45716,2,Valid,45641
recclass,45716,466,L6,8285
mass,45585,12576,1.3,171
fall,45716,2,Found,44609
year,45425,266,2003-01-01T00:00:00.000,3323
reclat,38401,12738,0.000000,6438
reclong,38401,14640,0.000000,6214
geolocation.latitude,38401,12738,0.0,6438


In [98]:
# 4.2 Convert necessary columns to appropriate data types

# df['mass'] = pd.to_numeric(df['mass (g)'], errors='coerce')
df['mass'] = pd.to_numeric(df['mass'], errors='coerce')
# df['year'] = pd.to_datetime(df['year'], errors='coerce').dt.year
df['year'] = df['year'].str.split('-', expand = True)[0]
df['reclat'] = pd.to_numeric(df['reclat'], errors='coerce')
df['reclong'] = pd.to_numeric(df['reclong'], errors='coerce')

In [99]:
# Display first five rows
df.head()

,name,id,nametype,recclass,mass,fall,year,reclat,reclong,geolocation.latitude,geolocation.longitude,:@computed_region_cbhk_fwbd,:@computed_region_nnqa_25f4
0,Aachen,1,Valid,L5,21.0,Fell,1880,50.77500,6.08333,50.775,6.08333,NaN,NaN
1,Aarhus,2,Valid,H6,720.0,Fell,1951,56.18333,10.23333,56.18333,10.23333,NaN,NaN
2,Abee,6,Valid,EH4,107000.0,Fell,1952,54.21667,-113.00000,54.21667,-113.0,NaN,NaN
3,Acapulco,10,Valid,Acapulcoite,1914.0,Fell,1976,16.88333,-99.90000,16.88333,-99.9,NaN,NaN
4,Achiras,370,Valid,L6,780.0,Fell,1902,-33.16667,-64.95000,-33.16667,-64.95,NaN,NaN


In [100]:
# list columns

list(df.columns)

['name',
 'id',
 'nametype',
 'recclass',
 'mass',
 'fall',
 'year',
 'reclat',
 'reclong',
 'geolocation.latitude',
 'geolocation.longitude',
 ':@computed_region_cbhk_fwbd',
 ':@computed_region_nnqa_25f4']

In [101]:
# df['reclat'].unique()

In [102]:
# Drop rows with missing values in critical columns (Delete all NaN values (mass and year and reclat/reclon)
df_cleaned = df.dropna(subset=['name', 'id', 'mass', 'year', 'reclat', 'reclong'], inplace = True)

# Drop the original mass column
# df_cleaned = df_cleaned.drop(columns=['mass (g)'])

In [103]:
# Drop columns with @ symbol
df_cleaned = df.drop(columns = ['id', ':@computed_region_cbhk_fwbd', ':@computed_region_nnqa_25f4'], axis = 1) 

In [120]:
# Convert 'year' column to integers
df_cleaned['year'] = df_cleaned['year'].astype(int)

In [121]:
# Display the cleaned dataframe
display(df_cleaned.head())
display(df_cleaned.info())

,name,nametype,recclass,mass,fall,year,reclat,reclong,geolocation.latitude,geolocation.longitude
0,Aachen,Valid,L5,21.0,Fell,1880,50.77500,6.08333,50.775,6.08333
1,Aarhus,Valid,H6,720.0,Fell,1951,56.18333,10.23333,56.18333,10.23333
2,Abee,Valid,EH4,107000.0,Fell,1952,54.21667,-113.00000,54.21667,-113.0
3,Acapulco,Valid,Acapulcoite,1914.0,Fell,1976,16.88333,-99.90000,16.88333,-99.9
4,Achiras,Valid,L6,780.0,Fell,1902,-33.16667,-64.95000,-33.16667,-64.95


<class 'pandas.core.frame.DataFrame'>
Index: 38115 entries, 0 to 45715
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   name                   38115 non-null  object 
 1   nametype               38115 non-null  object 
 2   recclass               38115 non-null  object 
 3   mass                   38115 non-null  float64
 4   fall                   38115 non-null  object 
 5   year                   38115 non-null  int32  
 6   reclat                 38115 non-null  float64
 7   reclong                38115 non-null  float64
 8   geolocation.latitude   38115 non-null  object 
 9   geolocation.longitude  38115 non-null  object 
dtypes: float64(3), int32(1), object(6)
memory usage: 3.1+ MB


None

In [122]:
# Check for null values
df_cleaned.isnull().sum()


name                     0
nametype                 0
recclass                 0
mass                     0
fall                     0
year                     0
reclat                   0
reclong                  0
geolocation.latitude     0
geolocation.longitude    0
dtype: int64

In [123]:
# Delete all data points with 0 lat and 0 long

cleaned_df = df_cleaned[(df_cleaned['reclat'] != 0) & (df_cleaned['reclong'] != 0)]

cleaned_df[['reclat','reclong']].describe().T

,count,mean,std,min,25%,50%,75%,max
reclat,31705.0,-47.602009,46.714171,-87.36667,-79.68333,-72.00000,18.39317,81.16667
reclong,31705.0,73.452607,83.419083,-165.43333,26.00000,57.14703,159.41482,178.20000


In [128]:
# Check for outlier years
print('--------------- Years after date --------------------------')
print([year for year in cleaned_df['year'].unique() if year > 2010])
print('--------------- Years Sorted Desc --------------------------')
print(cleaned_df['year'].sort_values(ascending=False))

--------------- Years after date --------------------------
[2012, 2011, 2013]
--------------- Years Sorted Desc --------------------------
194      2013
22890    2012
17260    2012
31062    2012
3425     2012
         ... 
5365     1576
283      1492
856      1491
278      1400
704       861
Name: year, Length: 31705, dtype: int32


In [133]:
# Meteorite reclassifications

cleaned_df.recclass.replace(to_replace=['Acapulcoite', 'Acapulcoite/Lodranite', 'Acapulcoite/lodranite',
           'Lodranite','Lodranite-an','Winonaite','Achondrite-prim'],value='PA',inplace=True)

cleaned_df.recclass.replace(to_replace=['Angrite', 'Aubrite','Aubrite-an','Ureilite', 'Ureilite-an','Ureilite-pmict',
           'Brachinite','Diogenite', 'Diogenite-an', 'Diogenite-olivine', 'Diogenite-pm',
           'Eucrite', 'Eucrite-Mg rich', 'Eucrite-an', 'Eucrite-br','Eucrite-cm',
           'Eucrite-mmict', 'Eucrite-pmict', 'Eucrite-unbr','Howardite'],value='AA',inplace=True)

cleaned_df.recclass.replace(to_replace=['Lunar', 'Lunar (anorth)', 'Lunar (bas. breccia)',
           'Lunar (bas/anor)', 'Lunar (bas/gab brec)', 'Lunar (basalt)',
           'Lunar (feldsp. breccia)', 'Lunar (gabbro)', 'Lunar (norite)'],value='Lu',inplace=True)

cleaned_df.recclass.replace(to_replace=['Martian', 'Martian (OPX)','Martian (chassignite)', 'Martian (nakhlite)',
           'Martian (shergottite)','Martian (basaltic breccia)'],value='Ma',inplace=True)

cleaned_df.recclass.replace(to_replace=['C','C2','C4','C4/5','C6','C1-ung', 'C1/2-ung','C2-ung',
           'C3-ung', 'C3/4-ung','C4-ung','C5/6-ung',
           'CB', 'CBa', 'CBb', 'CH/CBb', 'CH3', 'CH3 ', 'CI1', 'CK', 'CK3',
           'CK3-an', 'CK3.8', 'CK3/4', 'CK4', 'CK4-an', 'CK4/5', 'CK5',
           'CK5/6', 'CK6', 'CM', 'CM-an', 'CM1', 'CM1/2', 'CM2', 'CM2-an',
           'CO3', 'CO3 ', 'CO3.0', 'CO3.1', 'CO3.2', 'CO3.3', 'CO3.4', 'CO3.5',
           'CO3.6', 'CO3.7', 'CO3.8', 'CR', 'CR-an', 'CR1', 'CR2', 'CR2-an',
           'CV2', 'CV3', 'CV3-an','CR7'],value='CC',inplace=True)

cleaned_df.recclass.replace(to_replace=['OC', 'OC3','H', 'H(5?)', 'H(?)4', 'H(L)3', 'H(L)3-an', 'H-an','H-imp melt',
           'H-melt rock', 'H-metal', 'H/L3', 'H/L3-4', 'H/L3.5',
           'H/L3.6', 'H/L3.7', 'H/L3.9', 'H/L4', 'H/L4-5', 'H/L4/5', 'H/L5',
           'H/L6', 'H/L6-melt rock', 'H/L~4', 'H3', 'H3 ', 'H3-4', 'H3-5',
           'H3-6', 'H3-an', 'H3.0', 'H3.0-3.4', 'H3.1', 'H3.10', 'H3.2',
           'H3.2-3.7', 'H3.2-6', 'H3.2-an', 'H3.3', 'H3.4', 'H3.4-5',
           'H3.4/3.5', 'H3.5', 'H3.5-4', 'H3.6', 'H3.6-6', 'H3.7', 'H3.7-5',
           'H3.7-6', 'H3.7/3.8', 'H3.8', 'H3.8-4', 'H3.8-5', 'H3.8-6',
           'H3.8-an', 'H3.8/3.9', 'H3.8/4', 'H3.9', 'H3.9-5', 'H3.9-6',
           'H3.9/4', 'H3/4', 'H4', 'H4 ', 'H4(?)', 'H4-5', 'H4-6', 'H4-an',
           'H4/5', 'H4/6', 'H5', 'H5 ', 'H5-6', 'H5-7', 'H5-an',
           'H5-melt breccia', 'H5/6', 'H6', 'H6 ', 'H6-melt breccia', 'H6/7',
           'H7', 'H?','H~4', 'H~4/5', 'H~5', 'H~6','L', 'L(?)3',
           'L(H)3', 'L(LL)3', 'L(LL)3.05', 'L(LL)3.5-3.7', 'L(LL)5', 'L(LL)6',
           'L(LL)~4', 'L-imp melt', 'L-melt breccia', 'L-melt rock', 'L-metal',
           'L/LL', 'L/LL(?)3', 'L/LL-melt rock', 'L/LL3', 'L/LL3-5', 'L/LL3-6',
           'L/LL3.10', 'L/LL3.2', 'L/LL3.4', 'L/LL3.5', 'L/LL3.6/3.7', 'L/LL4',
           'L/LL4-6', 'L/LL4/5', 'L/LL5', 'L/LL5-6', 'L/LL5/6', 'L/LL6',
           'L/LL6-an', 'L/LL~4', 'L/LL~5', 'L/LL~6', 'L3', 'L3-4', 'L3-5',
           'L3-6', 'L3-7', 'L3.0', 'L3.0-3.7', 'L3.0-3.9', 'L3.05', 'L3.1',
           'L3.10', 'L3.2', 'L3.2-3.5', 'L3.2-3.6', 'L3.3', 'L3.3-3.5',
           'L3.3-3.6', 'L3.3-3.7', 'L3.4', 'L3.4-3.7', 'L3.5', 'L3.5-3.7',
           'L3.5-3.8', 'L3.5-3.9', 'L3.5-5', 'L3.6', 'L3.6-4', 'L3.7',
           'L3.7-3.9', 'L3.7-4', 'L3.7-6', 'L3.7/3.8', 'L3.8', 'L3.8-5',
           'L3.8-6', 'L3.8-an', 'L3.9', 'L3.9-5', 'L3.9-6', 'L3.9/4', 'L3/4',
           'L4', 'L4 ', 'L4-5', 'L4-6', 'L4-an', 'L4-melt rock', 'L4/5', 'L5',
           'L5 ', 'L5-6', 'L5-7', 'L5/6', 'L6', 'L6 ', 'L6-melt breccia',
           'L6-melt rock', 'L6/7', 'L7', 'LL', 'LL(L)3', 'LL-melt rock', 'LL3',
           'LL3-4', 'LL3-5', 'LL3-6', 'LL3.0', 'LL3.00', 'LL3.1', 'LL3.1-3.5',
           'LL3.10', 'LL3.15', 'LL3.2', 'LL3.3', 'LL3.4', 'LL3.5', 'LL3.6',
           'LL3.7', 'LL3.7-6', 'LL3.8', 'LL3.8-6', 'LL3.9', 'LL3.9/4', 'LL3/4',
           'LL4', 'LL4-5', 'LL4-6', 'LL4/5', 'LL4/6', 'LL5', 'LL5-6', 'LL5-7',
           'LL5/6', 'LL6', 'LL6 ', 'LL6(?)', 'LL6/7', 'LL7', 'LL7(?)',
           'LL<3.5', 'LL~3', 'LL~4', 'LL~4/5', 'LL~5', 'LL~6',
           'L~3', 'L~4', 'L~5', 'L~6','Relict H','Relict OC','LL-melt breccia','H-melt breccia',
                                 'L5-melt breccia','LL-imp melt','H3.05','LL6-melt breccia',
                                 'LL3.05','H4-melt breccia','LL3.8-4','L3.00','L~4-6','LL6-an',
                                 'L4-melt breccia','LL(L)3.1','L3-melt breccia','H3.15'],value='OC',inplace=True)

cleaned_df.recclass.replace(to_replace=['EH','EH-imp melt', 'EH3', 'EH3/4-an', 'EH4', 'EH4/5', 'EH5', 'EH6',
           'EH6-an', 'EH7', 'EH7-an', 'EL3', 'EL3/4', 'EL4', 'EL4/5', 'EL5',
           'EL6', 'EL6 ', 'EL6/7', 'EL7','E','E3','E4', 'E5','E6','EL-melt rock'],value='EC',inplace=True)

cleaned_df.recclass.replace(to_replace=['K', 'K3','R', 'R3', 'R3-4', 'R3-5', 'R3-6', 'R3.4', 'R3.5-6',
           'R3.6', 'R3.7', 'R3.8', 'R3.8-5', 'R3.8-6', 'R3.9', 'R3/4', 'R4',
           'R4/5', 'R5', 'R6','R3.5-4'],value='AOC',inplace=True)

cleaned_df.recclass.replace(to_replace=['Pallasite', 'Pallasite, PES','Pallasite, PMG',
           'Pallasite, PMG-an', 'Pallasite, ungrouped',
           'Pallasite?'],value='P',inplace=True)

cleaned_df.recclass.replace(to_replace=['Mesosiderite', 'Mesosiderite-A','Mesosiderite-A1',
           'Mesosiderite-A2', 'Mesosiderite-A3','Mesosiderite-A3/4',
           'Mesosiderite-A4', 'Mesosiderite-B','Mesosiderite-B1',
           'Mesosiderite-B2', 'Mesosiderite-B4','Mesosiderite-C',
           'Mesosiderite-C2', 'Mesosiderite-an','Mesosiderite?'],value='MG',inplace=True)

cleaned_df.recclass.replace(to_replace=['Iron, IC', 'Iron, IC-an', 'Iron, IIAB', 'Iron, IIAB-an',
           'Iron, IIC', 'Iron, IID', 'Iron, IID-an','Iron, IIF', 'Iron, IIG',
           'Iron, IIIAB', 'Iron, IIIAB-an', 'Iron, IIIAB?', 'Iron, IIIE',
           'Iron, IIIE-an', 'Iron, IIIF', 'Iron, IVA', 'Iron, IVA-an',
           'Iron, IVB'],value='MIM',inplace=True)

cleaned_df.recclass.replace(to_replace=['Iron, IAB complex', 'Iron, IAB-MG','Iron, IAB-an', 'Iron, IAB-sHH',
           'Iron, IAB-sHL', 'Iron, IAB-sLH','Iron, IAB-sLL', 'Iron, IAB-sLM',
           'Iron, IAB-ung', 'Iron, IAB?','Iron, IIE',
           'Iron, IIE-an', 'Iron, IIE?'],value='NMIM',inplace=True)

cleaned_df.recclass.replace(to_replace=['Iron','Iron?','Relict iron','Chondrite-fusion crust',
           'Fusion crust','Impact melt breccia',
           'Enst achon-ung','Iron, ungrouped','Stone-uncl', 'Stone-ung',
           'Unknown','Achondrite-ung','Chondrite-ung',
           'Enst achon','E-an',  'E3-an',  'E5-an','Howardite-an','C3.0-ung','Iron, IAB-sHL-an'],value='UN',inplace=True)

C:\Users\milli\AppData\Local\Temp\ipykernel_18480\374130019.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  cleaned_df.recclass.replace(to_replace=['Acapulcoite', 'Acapulcoite/Lodranite', 'Acapulcoite/lodranite',
C:\Users\milli\AppData\Local\Temp\ipykernel_18480\374130019.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_df.reccl

In [134]:
# Summary of reclassification counts
cleaned_df['recclass'].value_counts()

recclass
OC      28247
AA        828
CC        825
MIM       532
EC        385
UN        235
NMIM      228
MG        121
Lu        100
P          64
PA         58
Ma         44
AOC        38
Name: count, dtype: int64

In [135]:
# check the first five rows()
cleaned_df.head()


,name,nametype,recclass,mass,fall,year,reclat,reclong,geolocation.latitude,geolocation.longitude
0,Aachen,Valid,OC,21.0,Fell,1880,50.77500,6.08333,50.775,6.08333
1,Aarhus,Valid,OC,720.0,Fell,1951,56.18333,10.23333,56.18333,10.23333
2,Abee,Valid,EC,107000.0,Fell,1952,54.21667,-113.00000,54.21667,-113.0
3,Acapulco,Valid,PA,1914.0,Fell,1976,16.88333,-99.90000,16.88333,-99.9
4,Achiras,Valid,OC,780.0,Fell,1902,-33.16667,-64.95000,-33.16667,-64.95


In [136]:
# Create column class_name with a reclassification of the reclass column

meteorite_class_dict = {
    'CC': 'Carbonaceous Chondrite',
    'OC': 'Ordinary Chondrite',
    'Ma': 'Martian',
    'AA': 'Asteroidal Achondrites',
    'PA': 'Primitive Achondrites',
    'Lu': 'Lunar',
    'EC': 'Enstatite Chondrite',
    'AOC': 'Other Chondrite Groups',
    'P': 'Pallasites',
    'MG': 'Mesosiderite Group',
    'MIM': 'Magmatic Iron Meteorite Groups',
    'NMIM': 'Non-magmatic or Primitive Iron Meteorite Groups',
    'UN': 'Not Enough Information (Ungrouped or Unknown)'
}
# meteorite_class_dict



In [5]:
# 4.3 Check the unique values in the recclass column

toclassify = [x for x in df_cleaned['recclass'].unique()]
print(toclassify)

['L5', 'H6', 'EH4', 'Acapulcoite', 'L6', 'LL3-6', 'H5', 'L', 'Diogenite-pm', 'H4', 'H', 'Iron, IVA', 'CR2-an', 'LL5', 'CI1', 'L/LL4', 'Eucrite-mmict', 'CV3', 'Ureilite-an', 'Stone-uncl', 'L3', 'Angrite', 'LL6', 'L4', 'Aubrite', 'Iron, IIAB', 'Iron, IAB-sLL', 'Iron, ungrouped', 'CM2', 'Mesosiderite-A1', 'LL4', 'C2-ung', 'LL3.8', 'Howardite', 'Eucrite-pmict', 'Diogenite', 'LL3.15', 'LL3.9', 'Iron, IAB-MG', 'H/L3.9', 'Eucrite', 'H4-an', 'L/LL6', 'Iron, IIIAB', 'OC', 'H/L4', 'H4-5', 'L3.7', 'LL3.4', 'Martian (chassignite)', 'EL6', 'H3.8', 'H3-5', 'H5-6', 'Mesosiderite', 'H5-7', 'L3-6', 'H4-6', 'Ureilite', 'Mesosiderite-A3/4', 'CO3.3', 'H3', 'EH3/4-an', 'Iron, IIE', 'L/LL5', 'H3.7', 'CBa', 'H4/5', 'H3/4', 'H?', 'H3-6', 'L3.4', 'Iron, IAB-sHL', 'L3.7-6', 'Iron, IID', 'EH7-an', 'CR2', 'CO3.2', 'K3', 'L5/6', 'CK4', 'Iron, IIE-an', 'L3.6', 'Iron', 'LL3.2', 'CO3.5', 'Lodranite', 'Mesosiderite-A3', 'L3-4', 'H5/6', 'Pallasite, PMG', 'Eucrite-cm', 'Pallasite', 'L5-6', 'CO3.6', 'Martian (nakhlite)',

In [6]:
# Step 4.4 Define mapping for recclass to broader categories
# Define the broad categories for recclass
rocky_material = [
    'L5', 'H6', 'EH4', 'Acapulcoite', 'L6', 'LL3-6', 'H5', 'L', 'Diogenite-pm', 'H4', 'H', 'CR2-an', 
    'LL5', 'CI1', 'L/LL4', 'Eucrite-mmict', 'CV3', 'Ureilite-an', 'Stone-uncl', 'L3', 'Angrite', 
    'LL6', 'L4', 'Aubrite', 'CM2', 'LL4', 'C2-ung', 'LL3.8', 'Howardite', 'Eucrite-pmict', 'Diogenite', 
    'LL3.15', 'LL3.9', 'Eucrite', 'H4-an', 'L/LL6', 'OC', 'H/L3.9', 'H4-5', 'L3.7', 'LL3.4', 
    'Martian (chassignite)', 'EL6', 'H3.8', 'H3-5', 'H5-6', 'Mesosiderite', 'H5-7', 'L3-6', 'H4-6', 
    'Ureilite', 'Mesosiderite-A3/4', 'CO3.3', 'H3', 'EH3/4-an', 'H3.7', 'CBa', 'H4/5', 'H3/4', 'H?', 
    'H3-6', 'L3.4', 'L3.7-6', 'EH7-an', 'CR2', 'CO3.2', 'K3', 'L5/6', 'CK4', 'Iron, IIE-an', 'L3.6', 
    'LL3.2', 'CO3.5', 'Lodranite', 'Mesosiderite-A3', 'L3-4', 'H5/6', 'Eucrite-cm', 'Martian (nakhlite)', 
    'LL3.6', 'C3-ung', 'H3-4', 'CO3.4', 'EH3', 'Winonaite', 'LL', 'Eucrite-br', 'R3.8-6', 'L4-6', 'EH5', 
    'LL3.00', 'H3.4', 'Martian (shergottite)', 'Achondrite-ung', 'LL3.3', 'C', 'H/L3.6', 'LL7', 
    'LL4-6', 'CO3.7', 'L/LL6-an', 'Pallasite, PMG-an', 'H3.9/4', 'L3.8', 'LL5-6', 'LL3.8-6', 'L3.9', 
    'L4-5', 'L3-5', 'LL4/5', 'L4/5', 'H3.9', 'H3.6-6', 'H3.8-5', 'H3.8/4', 'H3.9-5', 'CH3', 'R3.8-5', 
    'L3.9/4', 'E4', 'CO3', 'Chondrite-ung', 'H~5', 'H~6', 'L/LL3.10', 'EL5', 'LL3', 'L~6', 'L~3', 'H~4', 
    'L(LL)3.5-3.7', 'H3.6', 'L3.4-3.7', 'L3.5', 'CM1/2', 'Martian (OPX)', 'Brachinite', 'LL7(?)', 'LL6(?)', 
    'Eucrite-Mg rich', 'H3.5-4', 'EL3', 'R3.6', 'H3.5', 'CM1', 'L/LL3', 'H7', 'L(?)3', 'L3.2', 'L3.7-3.9', 
    'Mesosiderite-B1', 'LL3.7', 'CO3.0', 'LL3.5', 'L3.7-4', 'Eucrite-unbr', 'CV3-an', 'Lunar (anorth)', 
    'L3.3', 'Lunar', 'CK5', 'H3.3', 'H3.7-6', 'E6', 'H3.0', 'H3.1', 'L3.0', 'L/LL3.4', 'C6', 'LL3.0', 
    'Lunar (gabbro)', 'R4', 'C4', 'H5-an', 'EH4/5', 'R3-6', 'L-imp melt', 'CK3', 'H3-an', 'L~5', 
    'Mesosiderite-an', 'Mesosiderite-A2', 'C4-ung', 'H(L)3-an', 'L(LL)3', 'H(L)3', 'R3', 'L7', 'CM-an', 
    'L/LL~6', 'L/LL~5', 'L~4', 'L/LL~4', 'LL(L)3', 'H3.2', 'L-melt breccia', 'H6-melt breccia', 
    'H5-melt breccia', 'H-melt rock', 'Eucrite-an', 'Lunar (bas/anor)', 'LL5/6', 'LL3/4', 'H3.4/3.5', 
    'Lunar (basalt)', 'H/L5', 'H(5?)', 'LL-imp melt', 'H~4/5', 'L6-melt breccia', 'L3.5-3.7', 'L3.3-3.7', 
    'L3.2-3.6', 'L3.3-3.6', 'Acapulcoite/Lodranite', 'Mesosiderite-B', 'CK5/6', 'L3.05', 'C2', 'C4/5', 
    'L/LL3.2', 'L3.5-5', 'L/LL(?)3', 'H4(?)', 'Relict iron', 'EL4/5', 'L5-7', 'Diogenite-an', 'CR1', 
    'H5 ', 'L5 ', 'H4 ', 'L4 ', 'E', 'L6 ', 'H3 ', 'LL6 ', 'H-metal', 'H6 ', 'L-metal', 'Mesosiderite-A4', 
    'L/LL5/6', 'H3.8-4', 'CBb', 'EL6/7', 'EL7', 'CH/CBb', 'CO3.8', 'H/L~4', 'R5', 'H4/6', 'H3.7-5', 
    'LL3.7-6', 'H3.7/3.8', 'L3.7/3.8', 'EH-imp melt', 'R', 'Aubrite-an', 'R6', 'L3.5-3.9', 'L3.2-3.5', 
    'L3.3-3.5', 'L3.0-3.7', 'E3-an', 'K', 'E3', 'CK4-an', 'L(LL)3.05', 'L3.10', 'Acapulcoite/lodranite', 
    'CB', 'LL-melt rock', 'EL-melt rock', 'EH6', 'L/LL4/5', 'L3.8-an', 'C5/6-ung', 'CV2', 'L3.8-6', 'H3.05', 
    'H3.10', 'Achondrite-prim', 'LL3.05', 'CK', 'Diogenite-olivine', 'LL3.8-4', 'C3.0-ung', 'LL-melt breccia', 
    'LL6-melt breccia', 'L5-melt breccia', 'CO3.1', 'R3-5', 'LL6-an', 'L4-melt breccia', 'Howardite-an', 
    'H4-melt breccia', 'Martian (basaltic breccia)', 'L3-melt breccia', 'L~4-6', 'LL~5', 'R3.5-4', 'CR-an', 
    'Eucrite-anom', 'L-melt rock', 'L/LL3.6/3.7', 'L/LL3.6-6', 'LL3.1', 'CK6', 'LL3.4-3.7', 'CK3-an', 
    'L3.8-5', 'H4-6 ', 'L3.9-6', 'L/LL3-6', 'LL3-5', 'H/L3', 'H/L3.5', 'L-melt breccia', 'L/LL3.9', 
    'L3.9-5', 'LL3.1-3.5', 'L3.6-4', 'LL3.0-3.4', 'LL3.1', 'L3.2-3.7', 'L/LL~3', 'LL3/6', 'R3/4', 'L3.4/3.5', 
    'H/L~4', 'LL~6', 'H/L3.2-3.7', 'H3.4-5', 'H/L~5', 'H/L3-6', 'H/L3.4', 'H/L3.5-3.7', 'H3.5-6', 'L~4-6', 
    'CO3.3 ', 'CO3.4 ', 'CO3.5 ', 'CO3.6 ', 'CO3.7 ', 'CO3.8 ', 'CR2 ', 'EH5 ', 'EH6 ', 'EL6 ', 'H~4 ', 'H~5 ', 
    'H~6 ', 'L~3 ', 'L~4 ', 'L~5 ', 'L~6 ', 'LL~6 ', 'L/LL3.10-6', 'C', 'CM', 'CO', 'CO3 ', 'CR', 'CR2', 'CV', 
    'CV3', 'E', 'EH', 'EH4', 'EH5', 'EH6', 'EL', 'EL6', 'K', 'K3', 'L', 'L/LL', 'L/LL3', 'L3', 'L4', 'L5', 'L6', 
    'LL', 'LL3', 'LL4', 'LL5', 'LL6', 'R', 'R3', 'R3-6', 'R4', 'R5', 'R6', 'EH3', 'EH4', 'EH5', 'EH6', 'EL4', 'EL5', 
    'EL6'
]
metallic_material = [
    'Iron, IVA', 'Iron, IIAB', 'Iron, IAB-sLL', 'Iron, ungrouped', 'Iron, IAB-MG', 'Iron, IIIAB', 'Iron, IIE', 
    'Iron, IAB-sHL', 'Iron, IID', 'Iron, IIIE', 'Iron, IVA-an', 'Iron, IIF', 'Iron, IIIAB-an', 'Iron, IIC',
    'Iron, IC', 'Iron, IID-an', 'Iron, IVB', 'Iron, IAB complex', 'Iron, IIIAB?', 'Iron, IIE-an', 'Iron, IC-an',
    'Iron, IIG', 'Iron, IIIF', 'Iron, IIIE-an', 'Iron, IAB-sLH', 'Iron, IAB-an', 'Iron, IAB-sHH', 'Iron, IAB-sLM',
    'Iron, IAB-ung', 'Iron, IAB?', 'Iron, IAB-sHL-an', 'Iron, IIE?', 'Iron'
]
mixtures = [
    'Pallasite', 'Mesosiderite-A1', 'Pallasite, PMG', 'Mesosiderite', 'Mesosiderite-A3/4', 'Pallasite, PMG-an',
    'Mesosiderite-A3', 'Pallasite?', 'Mesosiderite?', 'Mesosiderite-B2', 'Mesosiderite-A', 'Mesosiderite-B4', 
    'Mesosiderite-B', 'Pallasite, PES', 'Mesosiderite-C2', 'Mesosiderite-an', 'Mesosiderite-C', 'Pallasite, ungrouped'
]

def classify_recclass(recclass):
    if recclass in rocky_material:
        return 'Rocky'
    elif recclass in metallic_material:
        return 'Metallic'
    elif recclass in mixtures:
        return 'Mixtures'
    else:
        return 'Other'

# Apply the classification function
df_cleaned['meteorite_class'] = df_cleaned['recclass'].apply(classify_recclass)

# Display unique values in the new column
print(df_cleaned['meteorite_class'].unique())

df_cleaned.head()


['Rocky' 'Metallic' 'Mixtures' 'Other']


C:\Users\milli\AppData\Local\Temp\ipykernel_6260\3215058319.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['meteorite_class'] = df_cleaned['recclass'].apply(classify_recclass)


,name,id,nametype,recclass,mass,fall,year,reclat,reclong,geolocation,:@computed_region_cbhk_fwbd,:@computed_region_nnqa_25f4,meteorite_class
0,Aachen,1,Valid,L5,21.0,Fell,1880.0,50.77500,6.08333,"{'latitude': '50.775', 'longitude': '6.08333'}",NaN,NaN,Rocky
1,Aarhus,2,Valid,H6,720.0,Fell,1951.0,56.18333,10.23333,"{'latitude': '56.18333', 'longitude': '10.23333'}",NaN,NaN,Rocky
2,Abee,6,Valid,EH4,107000.0,Fell,1952.0,54.21667,-113.00000,"{'latitude': '54.21667', 'longitude': '-113.0'}",NaN,NaN,Rocky
3,Acapulco,10,Valid,Acapulcoite,1914.0,Fell,1976.0,16.88333,-99.90000,"{'latitude': '16.88333', 'longitude': '-99.9'}",NaN,NaN,Rocky
4,Achiras,370,Valid,L6,780.0,Fell,1902.0,-33.16667,-64.95000,"{'latitude': '-33.16667', 'longitude': '-64.95'}",NaN,NaN,Rocky


In [8]:
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Index: 38101 entries, 0 to 45715
Data columns (total 13 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   name                         38101 non-null  object 
 1   id                           38101 non-null  object 
 2   nametype                     38101 non-null  object 
 3   recclass                     38101 non-null  object 
 4   mass                         38101 non-null  float64
 5   fall                         38101 non-null  object 
 6   year                         38101 non-null  float64
 7   reclat                       38101 non-null  float64
 8   reclong                      38101 non-null  float64
 9   geolocation                  38101 non-null  object 
 10  :@computed_region_cbhk_fwbd  1647 non-null   object 
 11  :@computed_region_nnqa_25f4  1647 non-null   object 
 12  meteorite_class              38101 non-null  object 
dtypes: float64(4), object

In [ ]:
df

In [ ]:
# !pip install pandas psycopg2-binary SQLAlchemy


In [10]:
# # Step 5: Define SQLite database file path
# sqlite_db = 'meteorites.db'

# # Create a connection to SQLite database
# conn = sqlite3.connect(sqlite_db)

# # Create SQL table and insert DataFrame into it
# df_cleaned.to_sql('meteorites', conn, if_exists='replace', index=False)

# # Commit the changes and close the connection
# conn.commit()
# conn.close()

# print("SQLite database 'meteorites.db' and table 'meteorites' created successfully.")
